In [ ]:
# === CELL 1: GPU Check and Setup ===

# Check GPU availability
import tensorflow as tf
import os

print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Check GPU details (will only work in Colab)
print("\nGPU Details:")
# This command is for Colab environment
# !nvidia-smi

TensorFlow version: 2.19.0
GPU Available: []

GPU Details:


In [ ]:
# === CELL 2: Install Required Libraries and Imports ===

# Install additional libraries (will only work in Colab)
# !pip install -q kaggle opencv-python-headless scikit-learn matplotlib seaborn pillow

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import zipfile
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Deep Learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Sklearn for metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("✓ All libraries imported successfully!")

✓ All libraries imported successfully!


In [ ]:
!export KAGGLE_API_TOKEN=KGAT_0de04b77b71aa91707632f5cb5e93643


In [ ]:

# 1) تثبيت Kaggle
!pip install -q kaggle

# 2) إنشاء ملف kaggle.json بالطريقة التقليدية
import json
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username":"YOUR_KAGGLE_USERNAME","key":"YOUR_API_KEY"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

# 3) تنزيل وفك الضغط
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset --unzip -p brain_tumor_data
# === CELL 6: Set Paths and Count Images ===

# Set correct paths (adjust based on actual structure)
# The dataset typically has Training and Testing folders
train_dir = 'brain_tumor_data/Training'
test_dir = 'brain_tumor_data/Testing'

# Class names
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Count images per class
print("\n=== Training Set ===")
train_counts = {}
for class_name in class_names:
    class_path = os.path.join(train_dir, class_name)
    if os.path.exists(class_path):
        count = len(os.listdir(class_path))
        train_counts[class_name] = count
        print(f"{class_name}: {count} images")

print(f"\nTotal training images: {sum(train_counts.values())}")

print("\n=== Testing Set ===")
test_counts = {}
for class_name in class_names:
    class_path = os.path.join(test_dir, class_name)
    if os.path.exists(class_path):
        count = len(os.listdir(class_path))
        test_counts[class_name] = count
        print(f"{class_name}: {count} images")

print(f"\nTotal testing images: {sum(test_counts.values())}")


Dataset URL: https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset
License(s): CC0-1.0
 84% 125M/149M [00:00<00:00, 1.27GB/s]
100% 149M/149M [00:00<00:00, 1.01GB/s]

=== Training Set ===
glioma: 1321 images
meningioma: 1339 images
notumor: 1595 images
pituitary: 1457 images

Total training images: 5712

=== Testing Set ===
glioma: 300 images
meningioma: 306 images
notumor: 405 images
pituitary: 300 images

Total testing images: 1311


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 4571 images belonging to 4 classes.
Found 1141 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [ ]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# Freeze base model
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,121,476 (92.02 MB)

 Trainable params: 529,668 (2.02 MB)

 Non-trainable params: 23,591,808 (90.00 MB)

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-6)
]

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 1090s 8s/step - accuracy: 0.7979 - loss: 0.5758 - val_accuracy: 0.7528 - val_loss: 0.6530 - learning_rate: 1.0000e-04
Epoch 2/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 1108s 8s/step - accuracy: 0.8544 - loss: 0.3957 - val_accuracy: 0.8133 - val_loss: 0.5287 - learning_rate: 1.0000e-04
Epoch 3/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 1127s 8s/step - accuracy: 0.8803 - loss: 0.3204 - val_accuracy: 0.8422 - val_loss: 0.4321 - learning_rate: 1.0000e-04
Epoch 4/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 1078s 8s/step - accuracy: 0.8892 - loss: 0.2875 - val_accuracy: 0.8449 - val_loss: 0.4317 - learning_rate: 1.0000e-04
Epoch 5/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 1076s 8s/step - accuracy: 0.9017 - loss: 0.2715 - val_accuracy: 0.8501 - val_loss: 0.4186 - learning_rate: 1.0000e-04
Epoch 6/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 1093s 8s/step - accuracy: 0.9114 - loss: 0.2354 - val_accuracy: 0.8729 - val_loss: 0.3673 - learning_rate: 1.0000e-04
Epoch 7/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 1086s 8s/ste

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"ResNet50 Test Accuracy: {test_accuracy*100:.2f}%")


41/41 ━━━━━━━━━━━━━━━━━━━━ 238s 6s/step - accuracy: 0.8811 - loss: 0.3398
ResNet50 Test Accuracy: 90.77%


In [ ]:
base_model.trainable = True

# Freeze first 140 layers, fine-tune rest
for layer in base_model.layers[:140]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=callbacks
)
